# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

- Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:


In [1]:
# library for data analysis
import pandas as pd 

# library to handle requests
import requests 

# library to parse HTML documents
from bs4 import BeautifulSoup 

In [10]:
# get the response in the form of html
wikiurl=" https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table_class="wikitable sortable"
response=requests.get(wikiurl)
print("HTTP Status Code: " + str(response.status_code))
# print(response.text)

HTTP Status Code: 200


In [16]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
codestable=soup.find('table',{'class':table_class})
# print(codestable)

- convert table into  dataframe. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [17]:
df_codes=pd.read_html(str(codestable))

df_codes=pd.DataFrame(df_codes[0])

print(df_codes.head(10))

  Postal Code           Borough                                Neighbourhood
0         M1A      Not assigned                                 Not assigned
1         M2A      Not assigned                                 Not assigned
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
7         M8A      Not assigned                                 Not assigned
8         M9A         Etobicoke      Islington Avenue, Humber Valley Village
9         M1B       Scarborough                               Malvern, Rouge


In [18]:
# The code was removed by Watson Studio for sharing.

{'file_name': 'canada_codes.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'courseracapstoneproject-donotdelete-pr-hus2xobrw8txte',
 'asset_id': '7987bb01-6db2-408b-9216-6a1367561b65'}

In [ ]:
# Save Codes Dataframe into a CSV file for debugg 
from project_lib import Project

project = Project(project_id=project_id, project_access_token=project_token)
pc = project.project_context
project.save_data(data=df_codes.to_csv(index=False),file_name='canada_codes.csv',overwrite=True)

- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [6]:
df_codes = df_codes[df_codes["Borough"] != 'Not assigned']

- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
for index, row in df_codes.iterrows():
    neighbourhood = row['Neighbourhood']
    borough = row['Borough']
    # print(str(index) + "|" + neighbourhood + "|" + borough)
    if (str(neighbourhood).find('Not assigned') != -1 ): 
        df_codes[index]['Neighbourhood'] = df_codes[index]['Borough']

df_codes = df_codes.reset_index(drop=True)
df_codes.head(11)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [8]:
df_codes.shape

(103, 3)